### Need to fix spike_tools library, using notebook for exploration

In [2]:
%load_ext autoreload
%autoreload 2

from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)
import numpy as np
import pandas as pd

In [3]:
subject = "SA"
sess_name = "20180802"
spike_times = spike_general.get_spike_times(None, subject, sess_name, species_dir="/data")


In [4]:
spike_times

,UnitID,SpikeTime
0,0,207
1,0,436
2,0,1046
3,0,1094
4,0,1173
...,...,...
3210281,45,14170127
3210282,45,14172987
3210283,45,14173098
3210284,45,14175065


In [16]:
units = spike_general.list_session_units(None, subject, sess_name, species_dir="/data")

In [4]:
all_unit_info = spike_general.list_session_units(None, subject, sess_name, "/data")

In [5]:
filter_rows = all_unit_info
spike_times = filter_rows.apply(lambda x: spike_general._get_spike_timestamps(None, x), axis=1)

In [6]:
spike_times = spike_times.explode("SpikeTime")[["UnitID", "SpikeTime"]]

In [37]:
spike_times.reset_index().drop(columns="index")

,UnitID,SpikeTime
0,0,207
1,0,436
2,0,1046
3,0,1094
4,0,1173
...,...,...
3210281,45,14170127
3210282,45,14172987
3210283,45,14173098
3210284,45,14175065


In [14]:
spike_times_df = pd.DataFrame(spike_times, index=spike_times.index.values, columns=["SpikeTime"])
spike_times_df["UnitID"] = spike_times_df.index.values

In [19]:
SESS_SPIKES_PATH = "/data/patrick_res/firing_rates/{sess_name}_firing_rates_{pre_interval}_{event}_{post_interval}_{interval_size}_bins_1_smooth.pickle"
EVENT = "FeedbackOnset"  # event in behavior to align on
PRE_INTERVAL = 1300   # time in ms before event
POST_INTERVAL = 1500  # time in ms after event
INTERVAL_SIZE = 100  # size of interval in ms

def count_units(row):
    units = spike_general.list_session_units(None, subject, row.session_name, species_dir="/data")
    return len(units.UnitID.unique())

def count_units_from_fr(row):
    spikes_path = SESS_SPIKES_PATH.format(
        sess_name=row.session_name, 
        pre_interval=PRE_INTERVAL, 
        event=EVENT, 
        post_interval=POST_INTERVAL, 
        interval_size=INTERVAL_SIZE
    )
    frs = pd.read_pickle(spikes_path)
    return len(frs.UnitID.unique())

valid_sess = pd.read_pickle("/data/patrick_res/sessions/valid_sessions_rpe.pickle")
counts = valid_sess.apply(count_units_from_fr, axis=1)


In [20]:
sum(counts)

756